In [44]:
# === Cell 1: Environment Setup ===
# Mount Google Drive & install dependencies
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!pip install --quiet pymrmr swig

import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pymrmr

# Define shared paths
DRIVE_ROOT     = '/content/drive/Shareddrives/ECS289L'
MASTER_CSV     = f'{DRIVE_ROOT}/master_subjects.csv'
RAW_GEN_PATH   = f'{DRIVE_ROOT}/raw_genetics'
PLINK_PREFIX   = f'{RAW_GEN_PATH}/WGS_Omni25_BIN_wo_ConsentsIssues'
KEEP_LIST      = f'{RAW_GEN_PATH}/keep_genetics.txt'
SUBSET_PREFIX  = f'{RAW_GEN_PATH}/Omni25_subset'
QC_PREFIX      = f'{RAW_GEN_PATH}/Omni25_QC'
GEN_SMALL      = f'{RAW_GEN_PATH}/genetics_small'
GWAS_LIST_FILE = f'{RAW_GEN_PATH}/gwas_snps.txt'
EMB_DIR        = f'{DRIVE_ROOT}/embeddings/genetics'
os.makedirs(EMB_DIR, exist_ok=True)


Mounted at /content/drive


In [45]:
# === Cell 2A: Generate keep_genetics.txt from master_subjects & raw .fam ===
import pandas as pd
# Load the master 1,214‐RID list
master = pd.read_csv('/content/drive/Shareddrives/ECS289L/master_subjects.csv')
master_rids = set(master['RID'])

# Read the raw .fam to get (FID, IID) and extract numeric RID
fam = pd.read_csv(
    '/content/drive/Shareddrives/ECS289L/raw_genetics/WGS_Omni25_BIN_wo_ConsentsIssues.fam',
    sep='\s+',
    names=['FID','IID','PID','MID','SEX','PHENOTYPE']
)
fam['RID'] = fam['IID'].str.split('_').str[-1].astype(int)

# Keep only subject rows in master
keep_df = fam[fam['RID'].isin(master_rids)][['FID','IID']]
keep_path = '/content/drive/Shareddrives/ECS289L/raw_genetics/keep_genetics.txt'
keep_df.to_csv(keep_path, sep=' ', index=False, header=False)
print(f'✔️  keep_genetics.txt written with {len(keep_df)} entries → {keep_path}')


✔️  keep_genetics.txt written with 482 entries → /content/drive/Shareddrives/ECS289L/raw_genetics/keep_genetics.txt


In [46]:
# === Cell 2: PLINK Subset & QC ===

# Subset to master RIDs
!plink2 --bfile  $PLINK_PREFIX \
        --keep   $KEEP_LIST \
        --make-bed \
        --out    $SUBSET_PREFIX

# Apply QC filters
!plink2 --bfile  $SUBSET_PREFIX \
        --geno   0.02 \
        --mind   0.02 \
        --maf    0.01 \
        --hwe    1e-6 \
        --make-bed \
        --out    $QC_PREFIX

# Quick verification in Python
import subprocess
subs_fam = subprocess.check_output(['wc','-l', f'{QC_PREFIX}.fam']).decode().strip()
subs_bim = subprocess.check_output(['wc','-l', f'{QC_PREFIX}.bim']).decode().strip()
print('After QC →', subs_fam, 'subjects; ', subs_bim, 'SNPs')


PLINK v2.00a3 SSE4.2 (18 Feb 2022)             www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /content/drive/Shareddrives/ECS289L/raw_genetics/Omni25_subset.log.
Options in effect:
  --bfile /content/drive/Shareddrives/ECS289L/raw_genetics/WGS_Omni25_BIN_wo_ConsentsIssues
  --keep /content/drive/Shareddrives/ECS289L/raw_genetics/keep_genetics.txt
  --make-bed
  --out /content/drive/Shareddrives/ECS289L/raw_genetics/Omni25_subset

Start time: Sat Jun  7 10:26:32 2025
12977 MiB RAM detected; reserving 6488 MiB for main workspace.
Using up to 2 compute threads.
812 samples (364 females, 448 males; 812 founders) loaded from
/content/drive/Shareddrives/ECS289L/raw_genetics/WGS_Omni25_BIN_wo_ConsentsIssues.fam.
2379855 variants loaded from
/content/drive/Shareddrives/ECS289L/raw_genetics/WGS_Omni25_BIN_wo_ConsentsIssues.bim.
Note: No phenotype data present.
--keep: 482 samples remaining.
482 samples (200 females, 282 

In [47]:
# === Cell 2B: build gwas_snps.txt from Bellenguez-2022 association TSV ===
import os, pandas as pd

assoc_tsv = f'{RAW_GEN_PATH}/bellenguez2022_assoc.tsv'
gwas_txt  = f'{RAW_GEN_PATH}/gwas_snps.txt'

if not os.path.exists(assoc_tsv):
    raise FileNotFoundError(f'Upload bellenguez2022_assoc.tsv to {assoc_tsv}')

# read rsID + p-value
df = pd.read_csv(assoc_tsv, sep='\t', usecols=['SNPS','P-VALUE'])
df = df.rename(columns={'SNPS':'rsid', 'P-VALUE':'p_value'})

# sort & grab top-50 (it’s already genome-wide sig)
top50 = (
    df.sort_values('p_value')
      .drop_duplicates('rsid')
      .head(50)['rsid']
      .tolist()
)

with open(gwas_txt, 'w') as f:
    f.write('\n'.join(top50))

print(f'📝 wrote {len(top50)} SNPs → {gwas_txt}')


📝 wrote 50 SNPs → /content/drive/Shareddrives/ECS289L/raw_genetics/gwas_snps.txt


In [48]:
# === Cell 3: Genetics Embedding Pipeline (data-driven) ===
import os, pandas as pd
from tqdm.auto import tqdm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pymrmr, numpy as np

# 1) Load master list
master = pd.read_csv(MASTER_CSV)

# 2) Read the 50 Bellenguez rsIDs we just wrote
with open(GWAS_LIST_FILE) as f:
    gwas_snps = [l.strip() for l in f if l.strip()]
print(f"🔹 GWAS hits loaded: {len(gwas_snps)}")

# 3) Auto-discover APOE-region SNPs (chr19:44.9 Mb ±5 kb)
bim = pd.read_csv(f'{QC_PREFIX}.bim', sep='\s+',
                  names=['CHR','SNP','CM','POS','A1','A2'])
apoe_snps = (
    bim[(bim['CHR']==19) & bim['POS'].between(44908600,44909000)]
    ['SNP'].tolist()
)
print(f"🔹 APOE SNPs found: {len(apoe_snps)}  → {apoe_snps}")

# 4) Combine & dedupe
snps_all = list(dict.fromkeys(gwas_snps + apoe_snps))
print(f"⚙️  Total SNPs to extract: {len(snps_all)}")
if not snps_all:
    raise RuntimeError("❌ SNP list is empty—check gwas_snps.txt or APOE scan")

# 5) Write extract list for PLINK
with open(GWAS_LIST_FILE, 'w') as f:
    f.write('\n'.join(snps_all))

# 6) Export tiny dosage file
print("🔹 Exporting dosages with PLINK2…")
!plink2 --bfile  $QC_PREFIX \
        --extract $GWAS_LIST_FILE \
        --recode  A \
        --out     $GEN_SMALL

# 7) Load & impute
print("🔹 Loading & imputing dosages…")
df = pd.read_csv(f'{GEN_SMALL}.raw', sep='\s+')
snp_cols = df.columns[6:]
if len(snp_cols) == 0:
    raise RuntimeError("❌ No SNP columns in the .raw—PLINK extract failed")
for col in tqdm(snp_cols, desc="Imputing"):
    df[col] = df[col].fillna(df[col].mean())

# 8) One-hot encode
enc = OneHotEncoder(sparse_output=False, drop='first')
X_oh = enc.fit_transform(df[snp_cols])

# 9) Labels + mRMR
df['RID'] = df['IID'].str.split('_').str[-1].astype(int)
y = master.set_index('RID').loc[df['RID'], 'pMCI'].values
feat_df = pd.DataFrame(X_oh, index=df['RID'], columns=enc.get_feature_names_out())
feat_df['pMCI'] = y
k = min(100, feat_df.shape[1])
selected = pymrmr.mRMR(feat_df, 'MIQ', k)
print(f"   → selected {len(selected)} features")

# 10) Scale & save embeddings
os.makedirs(EMB_DIR, exist_ok=True)
scaler = StandardScaler()
X_sel = scaler.fit_transform(feat_df[selected].values).astype('float32')

records = []
for rid, vec in zip(feat_df.index, X_sel):
    path = f'{EMB_DIR}/{rid}.npy'
    np.save(path, vec)
    records.append({'RID': rid, 'path': path, 'mask': 1})

pd.DataFrame(records).to_csv(f'{EMB_DIR}/embeddings.csv', index=False)
print(f"✅ Embeddings saved → {EMB_DIR}/embeddings.csv")


🔹 GWAS hits loaded: 50


<ipython-input-48-e6c9ff3749bf>:16: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  bim = pd.read_csv(f'{QC_PREFIX}.bim', sep='\s+',


🔹 APOE SNPs found: 0  → []
⚙️  Total SNPs to extract: 50
🔹 Exporting dosages with PLINK2…
PLINK v2.00a3 SSE4.2 (18 Feb 2022)             www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /content/drive/Shareddrives/ECS289L/raw_genetics/genetics_small.log.
Options in effect:
  --bfile /content/drive/Shareddrives/ECS289L/raw_genetics/Omni25_QC
  --export A
  --extract /content/drive/Shareddrives/ECS289L/raw_genetics/gwas_snps.txt
  --out /content/drive/Shareddrives/ECS289L/raw_genetics/genetics_small

Start time: Sat Jun  7 10:27:09 2025
12977 MiB RAM detected; reserving 6488 MiB for main workspace.
Using up to 2 compute threads.
482 samples (200 females, 282 males; 482 founders) loaded from
/content/drive/Shareddrives/ECS289L/raw_genetics/Omni25_QC.fam.
1530033 variants loaded from
/content/drive/Shareddrives/ECS289L/raw_genetics/Omni25_QC.bim.
Note: No phenotype data present.
--extract: 10 variants remaining.
10 va

Imputing:   0%|          | 0/10 [00:00<?, ?it/s]

   → selected 25 features
✅ Embeddings saved → /content/drive/Shareddrives/ECS289L/embeddings/genetics/embeddings.csv
